# APEX Legend Analysis:
### -Project 2 (M9): Working with Web Data

Author: Ruoyu Chen

Github Repository: https://github.com/jc000222/Apexlegend_analysis

# Part 1: XML Parsing
First take a look at the robot.txt file

In [1]:
from src.my_package.xml_related import xml_load

website='https://apexlegendsstatus.com'
xml_parse = xml_load(website)

In [2]:
print(xml_parse.get_robotstxt())

User-agent: *
Allow: /
Sitemap: https://apexlegendsstatus.com/sitemap.xml


In [3]:
xml_parse.get_sitemap()

['https://apexlegendsstatus.com/sitemap.xml']

In [4]:
dataframes=xml_parse.get_sitemap_df()
dataframes[0]

,URL,Last Modified,priority
0,https://apexlegendsstatus.com/,2021-03-07T23:14:09+00:00,1.00
1,https://apexlegendsstatus.com/disclaimer,2021-03-07T23:14:09+00:00,0.80
2,https://apexlegendsstatus.com/how-to-fix/code-100,2021-03-07T23:14:09+00:00,0.80
3,https://apexlegendsstatus.com/how-to-fix/code-net,2021-03-07T23:14:09+00:00,0.80
4,https://apexlegendsstatus.com/how-to-fix/code-...,2021-03-07T23:14:09+00:00,0.80
...,...,...,...
62,https://apexlegendsstatus.com/interactive-map/...,2022-12-30T06:00:00+00:00,0.80
63,https://apexlegendsstatus.com/interactive-map/...,2022-12-30T06:00:00+00:00,0.80
64,https://apexlegendsstatus.com/interactive-map/...,2022-12-30T06:00:00+00:00,0.80
65,https://apexlegendsstatus.com/interactive-map/...,2022-12-30T06:00:00+00:00,0.80


# Part 3: Web Scraping

In [5]:
website='https://apexlegendsstatus.com'
print(xml_parse.get_robotstxt(website))

User-agent: *
Allow: /
Sitemap: https://apexlegendsstatus.com/sitemap.xml


In [6]:
from src.my_package.scraper import Scrap
scrape_webpage='https://apexlegendsstatus.com/game-stats/legends-pick-rates'
scraper=Scrap(scrape_webpage)

In [7]:
scraper.get_df()

,Legend,Pick Rate,Change,Avg Rank,Avg Level
0,Octane,9%,▲3.5%,Rookie 1,397
1,Conduit,8.6%,▼10.56%,Bronze 2,519
2,Pathfinder,8.3%,▲5.21%,Bronze 3,437
3,Wraith,7.6%,▲2.08%,Bronze 4,450
4,Lifeline,7.5%,▼6.29%,Rookie 1,413
5,Bangalore,7.1%,▲2.38%,Bronze 3,428
6,Horizon,5.9%,▲3.11%,Bronze 2,505
7,Bloodhound,5.3%,▲2.28%,Rookie 1,330
8,Fuse,5.1%,▲6.72%,Bronze 4,437
9,Revenant,4.2%,▼2.06%,Bronze 4,434
